In [141]:
%load_ext autoreload
%aimport -tf
%aimport data
%aimport model
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
import tensorflow as tf
import pandas as pd
import numpy as np
import data
import model

In [145]:
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
ds = data.load_reddit_data(
    max_document_length=40,
    min_frequency=50,
    test_size=5000)

In [ ]:
data.print_dataset_stats(ds)

In [148]:
BATCH_SIZE=64
train_input_fn, dev_input_fn = data.create_train_dev_input_fns(ds, batch_size=BATCH_SIZE)

In [149]:
def simple_rnn(sequences, sequence_lengths):
    cell = tf.nn.rnn_cell.GRUCell(256)
    sequence_list = tf.unpack(sequences, axis=1)
    return tf.nn.rnn(cell, sequence_list, dtype=tf.float32, sequence_length=sequence_lengths)

model_fn = model.create_lm(
    vocab_size=len(ds.vocab.vocabulary_),
    embedding_dim=64,
    rnn_fn=simple_rnn)

In [150]:
estimator = tf.contrib.learn.Estimator(model_fn=model_fn, model_dir="checkpoints/simple_rnn")
dev_monitor = tf.contrib.learn.monitors.ValidationMonitor(input_fn=dev_input_fn, every_n_steps=1000)
estimator.fit(input_fn=train_input_fn, steps=None, monitors=[dev_monitor])

INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 5.06666
INFO:tensorflow:Step 101: loss = 3.6658
INFO:tensorflow:Step 201: loss = 3.03505
INFO:tensorflow:Saving checkpoints for 300 into checkpoints/0/model.ckpt.
INFO:tensorflow:Step 301: loss = 3.25647
INFO:tensorflow:Step 401: loss = 3.43222
INFO:tensorflow:Step 501: loss = 3.4839
INFO:tensorflow:Saving checkpoints for 600 into checkpoints/0/model.ckpt.
INFO:tensorflow:Step 601: loss = 3.24548
INFO:tensorflow:Step 701: loss = 3.55857
INFO:tensorflow:Step 801: loss = 3.5469


KeyboardInterrupt: 

In [126]:
MAX_LEN=20
sentence = "I"
for i in range(MAX_LEN):
    print(sentence)
    x = np.array(list(ds.vocab.transform([sentence])))
    x_len = [i + 1]
    probs = estimator.predict(input_fn=lambda: {"x": sample, "x_len": [1]})
    next_word_idx = np.argmax(probs[i], axis=1)
    next_word = next(ds.vocab.reverse([next_word_idx]))
    sentence = sentence + " " + next_word
    if next_word == "EOS":
        break
    if i == MAX_LEN:
        break
    

I


NotFittedError: Couldn't find trained model at /var/folders/d3/jc_t7rns3_qf61016_0zhrh800c0pj/T/tmpc38nxszj.